# Import

In [11]:
import numpy as np
import pandas as pd
import random
import re

# Library Bantuan
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


# Library Sendiri
from libraryBantuan.katla import Katla

# Baca Data

In [3]:
daftar_kata = list(pd.read_csv('lima huruf.csv', header=None)[0])
print(f"Banyak Kata : {len(daftar_kata)}")

Banyak Kata : 6311


# Fungsi nebak

! = benar <br>
? = ada tapi salah tempat <br>
\* = tidak ada huruf tersebut


## Class GameKatla

In [4]:
class GameKatla():
    def __init__(self, daftar_kata):
        self.jumlah_tebakan = 0
        self.MENANG = "!!!!!"
        self.__jawaban = random.choice(daftar_kata)


    def tebak(self, tebakan):
        self.jumlah_tebakan += 1
        respon_pola = self.__evaluasi_tebakan(tebakan)

        if respon_pola == self.MENANG:                  # Sudah sama maka menang
            return ("selesai", self.jumlah_tebakan)
        if self.jumlah_tebakan == 6:                    # Sudah habis batas tebak dan masih salah
            return ("selesai", 10)
        

    def __evaluasi_tebakan(self, tebakan):
        list_jawaban = []
        respon_pola = ['*']*5
        for i in range(5):
            if self.__jawaban[i] == tebakan[i]:
                respon_pola[i] = '!'
            else:
                list_jawaban.append(self.__jawaban[i])
        
        for i in range(5):
            if respon_pola[i] == '!':
                continue
            if tebakan[i] in list_jawaban:
                respon_pola[i] = '?'
                list_jawaban.remove(tebakan[i])
        return ''.join(respon_pola)

        

# Program Utama

In [5]:
katla = Katla(daftar_kata)

katla.tebakan_selanjutnya("qudsi", "***?*")
katla.tebakan_selanjutnya("veste", "*?!**")
katla.tebakan_selanjutnya("faset", "*?!!*")
katla.tebakan_selanjutnya("absen", "!*!!!", True)

Banyak Kata : 1
['aksen']




'aksen'

In [11]:
katla.kandidat("veste", "*?!**", katla.daftar_kata)

[]

In [73]:
katla.kandidat("absen", "!*!!!", katla.daftar_kata)

['absen', 'aksen']

In [6]:
tes = Katla(daftar_kata)
tes.kandidat("absen", "!*!!!", daftar_kata)

a[^ b]sen


['aksen']

In [15]:
tebakan   = "veste"
pola      = "*?!**"
# jawaban = "aksen"

daf_kata = katla.daftar_kata
pola_regex = [r"\w"]*5
huruf_tanda_tanya = [tebakan[i] for i in range(5) if pola[i]=='?']
huruf_bintang = ''.join([tebakan[i] for i in range(5) if (pola[i]=='*') & (tebakan[i] not in huruf_tanda_tanya)])

# Membentuk pola regex
for i, p in enumerate(pola):
    if p == '!':
        pola_regex[i] = tebakan[i]
    elif p == '?':
        pola_regex[i] = f"[^ {tebakan[i]+huruf_bintang}]"
    elif p == '*':
        pola_regex[i] = f"[^ {huruf_bintang}]"


pola_regex = r"".join(pola_regex)
daf_kata = re.findall(pola_regex, ' '.join(daf_kata))
print(pola_regex)
# print(len(daf_kata))
# print(daf_kata)

# Mengambil kata2 yang ada huruf ?
list_harus_ada_huruf = [tebakan[i] for i in range(5) if pola[i]=='?']
new_daf_kata = []
for kata in daf_kata:
    kata_tanpa_benar = [kata[i] for i in range(5) if pola[i]!='!']
    sudah_benar = True
    for harus_ada_huruf in list_harus_ada_huruf:
        if harus_ada_huruf in kata_tanpa_benar:
            kata_tanpa_benar.remove(harus_ada_huruf)
        else:
            sudah_benar=False; break
    if sudah_benar:
        new_daf_kata.append(kata)

daf_kata = new_daf_kata
print(len(daf_kata))
print(daf_kata)

[^ vt][^ evt]s[^ vt][^ vt]
22
['absen', 'abses', 'aksen', 'aksep', 'akses', 'eksak', 'eksem', 'ekses', 'ekspo', 'eosen', 'kasep', 'kaspe', 'kosek', 'kosel', 'kosen', 'laser', 'maser', 'nasel', 'opsen', 'paser', 'poser', 'rasem']


# Cari kata terbaik

In [49]:


def cari_kata_terbaik(daftar_kata):
    def cari_semua_kemungkinan_pola():
        T = {'0' : '*', '1' : '?', '2' : '!'}
        list_pola = []
        for number in range(3**5):
            ternary=np.base_repr(number,base=3)
            pola = f"{ternary:05s}"
            list_pola.append(''.join([T[i] for i in pola]))
        return list_pola
    
    def P(x):
        return x / len(daftar_kata)
    
    semua_pola = cari_semua_kemungkinan_pola()
    katla = Katla(daftar_kata)
    kata_entropi = pd.DataFrame({'kata':[], 'entropi':[]})
    # Cari kata terbaik
    for kata in tqdm(daftar_kata):
        entropi = 0
        for pola in semua_pola:
            banyak = len(katla.kandidat(kata, pola, daftar_kata))
            pi = P(banyak)
            entropi += (pi*np.log2(pi)) if pi !=0 else 0
        entropi *= -1
        kata_entropi = pd.concat([kata_entropi, pd.DataFrame({'kata':[kata], 'entropi':[entropi]})], 
                                 ignore_index=True)
        kata_entropi = kata_entropi.sort_values(by='entropi', ascending=False)
        kata_entropi.to_csv('kata entropi.csv', index=False)

    # return kata_entropi

In [50]:
cari_kata_terbaik(daftar_kata)

100%|██████████| 6311/6311 [37:12<00:00,  2.83it/s]  


In [39]:
np.log2(0) * 0

nan